In [1]:
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

import numpy as np

import pydrake.all
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    LeafSystem, AbstractValue, PublishEvent, TriggerType, BasicVector, PiecewiseQuaternionSlerp,
    RandomGenerator, UniformlyRandomRotationMatrix, Simulator, FixedOffsetFrame, ConnectMeshcatVisualizer
    )

from panda_station import *

In [3]:
builder = DiagramBuilder()

problem_info = ProblemInfo("/home/agrobenj/drake-tamp/experiments/kitchen/problems/kitchen_problem.yaml")
station = problem_info.make_move_free_station()
builder.AddSystem(station)

plant, scene_graph = station.get_plant_and_scene_graph()
meshcat = ConnectMeshcatVisualizer(
    builder,
    scene_graph,
    output_port = station.GetOutputPort("query_object"),
    delete_prefix_on_load = True,
    zmq_url = zmq_url,
    #role = Role.kProximity
)
meshcat.load()
diagram = builder.Build()
simulator = Simulator(diagram)
simulator_context = simulator.get_context()
station_context = station.GetMyContextFromRoot(simulator_context)
plant_context = plant.GetMyContextFromRoot(simulator_context)
meshcat.start_recording()
simulator.AdvanceTo(1.0)
meshcat.stop_recording()
meshcat.publish_recording()
plant_context = station.GetSubsystemContext(plant, station_context)
scene_graph_context = station.GetSubsystemContext(scene_graph, station_context)
panda = station.get_panda()
hand = station.get_hand()
query_output_port = scene_graph.GetOutputPort("query")
query_object = query_output_port.Eval(scene_graph_context)
print(query_object.HasCollisions())

Building move_free station
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connected to meshcat-server.
False
